In [1]:
import os
import numpy as np
from scipy.optimize import minimize
from simsopt.geo import SurfaceRZFourier
from simsopt.objectives import SquaredFlux
from simsopt.objectives import QuadraticPenalty
from simsopt.geo import curves_to_vtk
from simsopt.field import BiotSavart, Current, coils_via_symmetries, Coil
from simsopt.geo import (
    CurveLength, CurveCurveDistance,
    MeanSquaredCurvature, LpCurveCurvature, CurveCWSFourier, ArclengthVariation, CurveXYZFourier
)

import matplotlib.pyplot as plt

In [2]:
wout = '../input.axiTorus_nfp3_QA_final'

ncoils = 1
order = 1 # order of dofs of cws curves
quadpoints = 300 #13 * order
ntheta = 50
nphi = 42
ext_via_normal_factor = 0.2565

In [3]:
s = SurfaceRZFourier.from_vmec_input(wout, range="half period", ntheta=ntheta, nphi=nphi)
s_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", ntheta=ntheta, nphi=int(nphi*2*s.nfp))
# CREATE COIL WINDING SURFACE SURFACE
cws = SurfaceRZFourier.from_vmec_input(wout, range="half period", ntheta=ntheta, nphi=nphi)
cws_full = SurfaceRZFourier.from_vmec_input(wout, range="full torus", ntheta=ntheta, nphi=int(nphi*2*s.nfp))

cws.extend_via_normal(ext_via_normal_factor)
cws_full.extend_via_normal(ext_via_normal_factor)

# CREATE CURVES + COILS     
base_curves = []
for i in range(ncoils):
    curve_cws = CurveCWSFourier(
        mpol=cws.mpol,
        ntor=cws.ntor,
        idofs=cws.x,
        quadpoints=quadpoints,
        order=order,
        nfp=cws.nfp,
        stellsym=cws.stellsym,
    )
    angle = (i + 0.5)*(2 * np.pi)/((2) * s.nfp * ncoils)
    curve_dofs = np.zeros(len(curve_cws.get_dofs()),)
    curve_dofs[0] = 1
    curve_dofs[2*order+2] = 0
    curve_dofs[2*order+3] = angle
    curve_cws.set_dofs(curve_dofs)
    curve_cws.fix(0)
    curve_cws.fix(2*order+2)
    base_curves.append(curve_cws)

    # saddle_coil_cws_outer = CurveCWSFourier(
    #     mpol=cws.mpol,
    #     ntor=cws.ntor,
    #     idofs=cws.x,
    #     quadpoints=quadpoints,
    #     order=order,
    #     nfp=cws.nfp,
    #     stellsym=cws.stellsym,
    # )
    # angle = (i + 0.5)*(2 * np.pi)/((2) * s.nfp * ncoils)
    # saddle_coil_dofs_outer = np.zeros(len(saddle_coil_cws_outer.get_dofs()),) 
    # saddle_coil_dofs_outer[order] = np.pi
    # saddle_coil_dofs_outer[2*order+3] = angle
    # saddle_coil_dofs_outer[2*order] = 1
    # saddle_coil_dofs_outer[2*order+5] = 0.4
    # saddle_coil_cws_outer.set_dofs(saddle_coil_dofs_outer)
    # saddle_coil_cws_outer.fix(0)
    # saddle_coil_cws_outer.fix(2*order+2)
    # base_curves.append(saddle_coil_cws_outer)

    # saddle_coil_cws_inner = CurveCWSFourier(
    #     mpol=cws.mpol,
    #     ntor=cws.ntor,
    #     idofs=cws.x,
    #     quadpoints=quadpoints,
    #     order=order,
    #     nfp=cws.nfp,
    #     stellsym=cws.stellsym,
    # )
    # # angle = (i + 0.5)*(2 * np.pi)/((2) * s.nfp * ncoils)
    # saddle_coil_dofs_inner = np.zeros(len(saddle_coil_cws_inner.get_dofs()),)
    # saddle_coil_dofs_inner[2*order+3] = angle
    # saddle_coil_dofs_inner[2*order] = 1
    # saddle_coil_dofs_inner[2*order+5] = 0.4
    # saddle_coil_cws_inner.set_dofs(saddle_coil_dofs_inner)
    # saddle_coil_cws_inner.fix(0)
    # saddle_coil_cws_inner.fix(2*order+2)
    # base_curves.append(saddle_coil_cws_inner)
    # print(angle)


base_currents = [Current(1)*1e5 for i in range(1*ncoils)]
print(base_curves)
print(s.nfp)


3


In [7]:


coils = coils_via_symmetries(base_curves, base_currents, s.nfp, True)
print(len(coils))

ax = s_full.plot(color="blue", close=True, show=False, engine="plotly", opacity=0.5)
ax.update_layout(showlegend=False)
cws_full.plot(ax=ax, color="lightblue", show=False, engine="plotly", close=True, opacity=0.5)
ax.update_layout(showlegend=False)
for coil in coils:
    coil.plot(ax=ax, color="green", show=False, engine="plotly", close=True)
# set box size
ax.update_layout(autosize=False, width=800, height=800)
# remove all bars 
ax.update_layout(showlegend=False)
ax.show()

6


In [5]:
OUT_DIR = "./saddlecoils/"
os.makedirs(OUT_DIR, exist_ok=True)

bs = BiotSavart(coils)
bs.set_points(s_full.gamma().reshape((-1, 3)))
curves = [c.curve for c in coils]
curves_to_vtk(curves, OUT_DIR + "curves_init")
curves_to_vtk(base_curves, OUT_DIR + "base_curves_init")
pointData = {"B_N": np.sum(bs.B().reshape((int(nphi*2*s_full.nfp), ntheta, 3)) * s_full.unitnormal(), axis=2)[:, :, None]}
s_full.to_vtk(OUT_DIR + "surf_init", extra_data=pointData)
cws_full.to_vtk(OUT_DIR + "cws_init")
